# R2R RAG Quickstart (Linux)

This notebook shows how to use R2R for Retrieval-Augmented Generation (RAG):
- Install the Python SDK
- Verify the API is reachable
- Ingest a sample document
- Run search and RAG
- Try the agent for deeper analysis

To run, start the R2R API separately in a Linux shell.

## Prerequisites (run in a separate terminal)

1) Create a Python 3.12 venv and install the server (from this repo):
```bash
python3.12 -m venv .venv
source .venv/bin/activate
pip install -e ./py
```

2) Configure your providers (OpenAI-compatible):
These examples mirror defaults in docker/compose.dev.yaml.
```bash
# Text generation (e.g., LM Studio / OpenAI-compatible)
export LMSTUDIO_API_BASE="http://localhost:8000/v1"  # from docker/compose.dev.yaml
export LMSTUDIO_API_KEY="123"  # example key used in compose.dev.yaml; replace as needed

# Embeddings (can be a different OpenAI-compatible base)
export OPENAI_API_BASE="http://athena.skhms.com/rse/embedding/qwen3-4b/v1"
export OPENAI_API_KEY="<embed-token>"  # set your key; compose.dev.yaml shows a sample

# Postgres (adjust as needed)
export R2R_POSTGRES_HOST=127.0.0.1  # from compose.dev.yaml
export R2R_POSTGRES_PORT=5432
export R2R_POSTGRES_USER=r2r
export R2R_POSTGRES_PASSWORD=r2rpassword
export R2R_POSTGRES_DBNAME=r2r
export R2R_PROJECT_NAME=r2r_local
```

Note: Ensure embedding dimensions in config match (see py/r2r/r2r.toml).

3) Start the API server:
```bash
export R2R_PORT=8002
python -m r2r.serve
# Server listens on http://0.0.0.0:8002
```

Then continue here in the notebook.

In [1]:
# Install the Python SDK for the client
%pip -q install r2r requests


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Configure providers and Postgres via Python env (this kernel only)
# If your R2R server runs in a separate process,
# set these in that process as well (compose/env).
import os

def setdefault_env(key, value):
    # if not os.environ.get(key):
    os.environ[key] = str(value)

# LM Studio / text generation (compose.dev.yaml defaults)
setdefault_env("LMSTUDIO_API_BASE", "http://localhost:8000/v1")
setdefault_env("LMSTUDIO_API_KEY", "123")  # example; replace with your token

# OpenAI-compatible embeddings
setdefault_env("OPENAI_API_BASE", "http://athena.skhms.com/rse/embedding/qwen3-4b/v1")
setdefault_env("OPENAI_API_KEY", "rse-s0HBWeEhWUXJx5LI3gNNtJWHdKBOePvRVuh8s3BplYRiT7VZ0N2aMej")


# Postgres
setdefault_env("R2R_POSTGRES_HOST", "127.0.0.1")
setdefault_env("R2R_POSTGRES_PORT", "5432")
setdefault_env("R2R_POSTGRES_USER", "r2r")
setdefault_env("R2R_POSTGRES_PASSWORD", "r2rpassword")
setdefault_env("R2R_POSTGRES_DBNAME", "r2r")
setdefault_env("R2R_PROJECT_NAME", "r2r_local")

print("Configured:")
for k in [
    "LMSTUDIO_API_BASE","LMSTUDIO_API_KEY",
    "OPENAI_API_BASE",
    "R2R_POSTGRES_HOST","R2R_POSTGRES_PORT","R2R_POSTGRES_DBNAME","R2R_PROJECT_NAME"
]:
    print(f" - {k}={os.environ.get(k)}")


Configured:
 - LMSTUDIO_API_BASE=http://localhost:8000/v1
 - LMSTUDIO_API_KEY=123
 - OPENAI_API_BASE=http://athena.skhms.com/rse/embedding/qwen3-4b/v1
 - R2R_POSTGRES_HOST=127.0.0.1
 - R2R_POSTGRES_PORT=5432
 - R2R_POSTGRES_DBNAME=r2r
 - R2R_PROJECT_NAME=r2r_local


In [3]:
# Point to your running R2R server
import os
BASE_URL = os.getenv('R2R_BASE_URL', 'http://localhost:8002')
print('Using BASE_URL =', BASE_URL)

Using BASE_URL = http://localhost:8002


In [4]:
# Sanity check: hit the OpenAPI spec
import requests, json
spec_url = BASE_URL + '/openapi_spec'
try:
    r = requests.get(spec_url, timeout=10)
    r.raise_for_status()
    print('OpenAPI available at', spec_url)
    # print a small part of the spec
    data = r.json() if r.headers.get('content-type','').startswith('application/json') else r.text
    if isinstance(data, dict):
        print('Title:', data.get('info', {}).get('title'))
    else:
        print(str(data)[:200] + '...')
except Exception as e:
    print('Failed to reach server:', e)
    print('Make sure the server is running as described above.')

OpenAPI available at http://localhost:8002/openapi_spec
Title: R2R Application API


In [5]:
# Initialize the client
from r2r import R2RClient
client = R2RClient(base_url=BASE_URL)
client

## Ingest a sample document

You can upload your own `.pdf`, `.txt`, `.md`, etc.
Below we create a small `.txt` file and ingest it.

In [6]:
# Create a small sample file
sample_path = 'sample_doc.txt'
with open(sample_path, 'w', encoding='utf-8') as f:
    f.write('DeepSeek R1 is a reasoning model. This file is a simple demo.\n')
    f.write('RAG combines retrieval with generation to produce grounded answers.')
print('Wrote', sample_path)

# Ingest the file
doc = client.documents.create(file_path=sample_path)
doc

Wrote sample_doc.txt
<Response [409 Conflict]>


R2RException: {'message': 'Document 40030f07-db03-59c3-ba46-7376251cd7b1 is currently ingesting with status augmenting.', 'error_type': 'R2RException'}

In [ ]:
# List your documents
docs = client.documents.list()
# len(docs)
# docs[0] if docs else None

## Search

Run a semantic or hybrid search over ingested content.

In [8]:
search_res = client.retrieval.search(query='What is RAG?')
search_res

<Response [500 Internal Server Error]>


R2RException: An error 'invalid input syntax for type vector: "[[-0.00037384033203125, -0.03173828125, -0.0126953125, 0.01104736328125, -0.0018157958984375, 0.0927734375, 0.030517578125, -0.00897216796875, 0.0211181640625, 0.04443359375, 0.033203125, -0.0238037109375, 0.005706787109375, -0.0023956298828125, 0.0625, -0.0079345703125, -0.002166748046875, -0.005859375, 0.01104736328125, -0.0027008056640625, -0.01226806640625, 0.044677734375, 0.0027923583984375, 0.04736328125, -0.005218505859375, -0.018798828125, -0.0244140625, -0.0225830078125, 0.037353515625, 0.0201416015625, 0.01226806640625, -0.036376953125, 0.078125, 0.037109375, 0.002593994140625, -0.0206298828125, 0.00019073486328125, -0.0086669921875, 0.0235595703125, 0.00726318359375, 0.01043701171875, -0.0128173828125, 0.0189208984375, 0.00921630859375, 0.00390625, 0.0184326171875, 0.000583648681640625, -0.0257568359375, -0.00604248046875, -0.007232666015625, 0.0084228515625, 0.000507354736328125, 0.01470947265625, -0.01806640625

## RAG (with citations)

Ask a question and let R2R retrieve + generate an answer grounded in your documents.

In [7]:
rag_res = client.retrieval.rag(query='Explain RAG briefly.')
rag_res

<Response [500 Internal Server Error]>


R2RException: An error '500: Internal RAG Error - invalid input syntax for type vector: "[[-2.0742416381835938e-05, -0.0264892578125, 0.00482177734375, 0.031005859375, -8.869171142578125e-05, 0.048095703125, 0.0341796875, -0.01318359375, 0.022705078125, 0.0478515625, 0.0228271484375, 0.004638671875, 0.000865936279296875, 0.00494384765625, 0.0167236328125, -0.0218505859375, -0.0126953125, 0.0208740234375, 0.013671875, -0.0072021484375, -0.005279541015625, 0.037353515625, 0.0078125, 0.064453125, -0.01416015625, -0.015625, -0.0206298828125, 0.0026702880859375, 0.02685546875, 0.0361328125, 0.0234375, -0.036376953125, 0.048828125, 0.03564453125, 0.00156402587890625, -0.0260009765625, -0.01068115234375, -0.007232666015625, 0.046630859375, 0.035888671875, 0.0031890869140625, -0.045166015625, 0.01422119140625, 0.01611328125, 0.0032958984375, 0.00604248046875, 0.00360107421875, -0.0294189453125, -0.0218505859375, -0.00958251953125, 0.00579833984375, 0.0103759765625, 0.004974365234375, -0.011657

## Agentic RAG

Use the conversational agent with retrieval tools for richer, multi-step answers.

In [ ]:
agent_res = client.retrieval.agent(
    message={"role": "user", "content": "Give a short analysis of RAG."},
    rag_tools=["search_file_knowledge", "get_file_content"],
)
agent_res

## Advanced: custom search settings

You can pass `search_mode` and `search_settings` to control hybrid search, filters, etc.

In [ ]:
advanced = client.retrieval.rag(
    query='What does the sample say about RAG?',
    search_mode='advanced',
    search_settings={
        "use_hybrid_search": True,
        "hybrid_settings": {
            "full_text_weight": 1.0,
            "semantic_weight": 5.0,
            "full_text_limit": 50,
            "rrf_k": 50
        }
    }
)
advanced